<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/analisis_comportamiento_usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Comportamiento de Usuarios con PySpark
Este notebook está diseñado para guiar paso a paso a los estudiantes en el análisis de comportamiento de usuarios en un entorno de comercio electrónico. Se utiliza un dataset público, se aplican transformaciones, se construyen perfiles de usuario y se realiza un clustering para identificar grupos de comportamiento.

## Objetivos
- Cargar y explorar un dataset de eventos de navegación.
- Transformar los datos y preparar características relevantes.
- Aplicar clustering para identificar tipos de usuarios.
- Analizar resultados y extraer conclusiones.

## Paso 1: Importación de librerías necesarias

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, avg
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt

**Explicación**: Se importan las librerías necesarias para manipulación de datos con PySpark, funciones SQL, herramientas de machine learning (clustering con KMeans) y visualización con matplotlib.

## Paso 2: Inicializar SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("UserBehaviorAnalysis") \
    .getOrCreate()

**Explicación**: Se inicia una sesión de Spark, que es necesaria para ejecutar cualquier operación con PySpark. Esta sesión permite crear y manipular DataFrames distribuidos.

## Paso 3: Cargar dataset
Se utiliza un subconjunto simulado del dataset de eventos de navegación. Si se tiene el dataset de Kaggle, se puede adaptar esta parte para leer desde CSV.

In [3]:
# Crear un dataframe simulado
data = [
    (1, 'view', 10), (1, 'cart', 0), (2, 'view', 5), (2, 'purchase', 1),
    (3, 'view', 7), (3, 'view', 2), (3, 'cart', 1), (4, 'view', 1),
    (4, 'cart', 1), (4, 'purchase', 1), (5, 'view', 12), (5, 'cart', 0)
]
columns = ['user_id', 'event_type', 'event_count']
df = spark.createDataFrame(data, columns)
df.show()

+-------+----------+-----------+
|user_id|event_type|event_count|
+-------+----------+-----------+
|      1|      view|         10|
|      1|      cart|          0|
|      2|      view|          5|
|      2|  purchase|          1|
|      3|      view|          7|
|      3|      view|          2|
|      3|      cart|          1|
|      4|      view|          1|
|      4|      cart|          1|
|      4|  purchase|          1|
|      5|      view|         12|
|      5|      cart|          0|
+-------+----------+-----------+



**Explicación**: Se crea un DataFrame simulado con datos de eventos de navegación de varios usuarios. Cada fila representa un evento con un tipo (vista, carrito, compra) y una cantidad asociada. Este conjunto simula un flujo real de interacciones de usuarios en una plataforma.

## Paso 4: Pivotar datos para análisis por usuario

In [4]:
pivot_df = df.groupBy("user_id").pivot("event_type").sum("event_count").na.fill(0)
pivot_df.show()

+-------+----+--------+----+
|user_id|cart|purchase|view|
+-------+----+--------+----+
|      5|   0|       0|  12|
|      1|   0|       0|  10|
|      3|   1|       0|   9|
|      2|   0|       1|   5|
|      4|   1|       1|   1|
+-------+----+--------+----+



**Explicación**: Se realiza una transformación tipo `pivot` para convertir los eventos en columnas y así tener una fila por usuario con sus respectivos conteos de cada tipo de evento. Esto facilita el análisis comparativo entre usuarios.

## Paso 5: Preparar datos para clustering con KMeans

In [6]:
assembler = VectorAssembler(inputCols=["view", "cart", "purchase"], outputCol="features")
dataset = assembler.transform(pivot_df)
dataset.select("user_id", "features").show(truncate=False)

+-------+--------------+
|user_id|features      |
+-------+--------------+
|5      |[12.0,0.0,0.0]|
|1      |[10.0,0.0,0.0]|
|3      |[9.0,1.0,0.0] |
|2      |[5.0,0.0,1.0] |
|4      |[1.0,1.0,1.0] |
+-------+--------------+



**Explicación**: Se utiliza `VectorAssembler` para combinar las columnas `view`, `cart` y `purchase` en un único vector de características, que es el formato requerido para aplicar algoritmos de machine learning en PySpark.

## Paso 6: Aplicar clustering con KMeans

In [7]:
kmeans = KMeans(k=2, seed=1)
modelo = kmeans.fit(dataset)
resultados = modelo.transform(dataset)
resultados.select("user_id", "view", "cart", "purchase", "prediction").show()

+-------+----+----+--------+----------+
|user_id|view|cart|purchase|prediction|
+-------+----+----+--------+----------+
|      5|  12|   0|       0|         0|
|      1|  10|   0|       0|         0|
|      3|   9|   1|       0|         0|
|      2|   5|   0|       1|         1|
|      4|   1|   1|       1|         1|
+-------+----+----+--------+----------+



**Explicación**: Se entrena un modelo KMeans con 2 clústeres sobre los vectores de características generados. Luego se aplica el modelo al dataset para predecir a qué grupo pertenece cada usuario. Esta agrupación ayuda a identificar diferentes patrones de comportamiento.

## Paso 7: Interpretación de resultados

In [8]:
agrupados = resultados.groupBy("prediction").agg(
    avg("view").alias("avg_views"),
    avg("cart").alias("avg_cart"),
    avg("purchase").alias("avg_purchase")
)
agrupados.show()

+----------+------------------+------------------+------------+
|prediction|         avg_views|          avg_cart|avg_purchase|
+----------+------------------+------------------+------------+
|         1|               3.0|               0.5|         1.0|
|         0|10.333333333333334|0.3333333333333333|         0.0|
+----------+------------------+------------------+------------+



**Explicación**: Se calculan los promedios de eventos por tipo en cada clúster generado por KMeans. Esto permite interpretar el significado de cada grupo, por ejemplo, usuarios que solo visualizan versus usuarios que también agregan al carrito y compran.

## Conclusiones
- Se identificaron diferentes grupos de usuarios basados en sus patrones de navegación.
- El modelo KMeans permite identificar grupos como compradores activos y visitantes pasivos.
- Este análisis puede ser usado para diseñar campañas de marketing personalizadas o detectar posibles abandonos de carrito.